In [1]:
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
import random
from colorama import Fore
from colorama import Style
import time
from datetime import timedelta

edgecount=383842 #number of ratings
usercount=278854 #number of users
bookcount=271361 #number of books

G=nx.Graph()
N=pd.read_csv('final.csv') #input weighted edgelist - column 1 - source, column 2 - target, column 3 - rating
Books=pd.read_csv('new_books.csv') #input books database

for i in range(edgecount): #forming weighted graph
    G.add_node(N.Book[i])
    G.add_node(N.User[i])
    G.add_edge(N.User[i],N.Book[i], weight=N.Rating[i])

In [80]:
print("You have three options:")
print("You can choose to get recommendations based on a book you are currently viewing or,")
print("You can choose to get recommendations based on a reading list that you will input or,")
print("You can choose to get recommendations based on a user")
print()

print("Do you choose to have recommendations given for one book that you are currently viewing? If Yes, hit 1. If No, hit 0")
flag1=int(input())
if flag1==1:
    print("input current book")
    current_node=int(input())
else:
    print("Do you choose to have recommendations given for a reading list that you will input? If Yes, hit 1. If No, hit 0")
    flag2=int(input())
    if flag2==1:
        print("Enter reading list and separate each entry by a space")
        reading_list=[int(x) for x in input().split()]
    else:
        print("Do you choose to have recommendations based on a particular user?")
        flag3=int(input())
        if flag3==1:
            print("Enter userID")
            current_user=int(input())


You have three options:
You can choose to get recommendations based on a book you are currently viewing or,
You can choose to get recommendations based on a reading list that you will input or,
You can choose to get recommendations based on a user

Do you choose to have recommendations given for one book that you are currently viewing? If Yes, hit 1. If No, hit 0
0
Do you choose to have recommendations given for a reading list that you will input? If Yes, hit 1. If No, hit 0
0
Do you choose to have recommendations based on a particular user?
1
Enter userID
11676


In [83]:
reco_number=10 #number of recommendations needed
Z=nx.Graph() #new graph
if flag2:
    if current_node not in reading_list:
        reading_list.append(current_node)

temp_user=list(set(N.User))
temp_user.sort()

In [84]:
#subgraph formation using G
wt=[]
row=[0]*(usercount+bookcount+1)
for i in range(usercount+bookcount+1):
    wt.append(row)
    
if flag1==1:
    reading_list=[current_node]
else:
    if flag3==1:
        reco_user=current_user
        if reco_user in G.nodes():
            avg_rating=G.degree(reco_user, weight='weight')/G.degree(reco_user)
            user_fav=[]
            for i in set(N.Book):
                if G.has_edge(reco_user, i):
                    if G[reco_user][i]['weight']>=min(avg_rating, 10):
                        user_fav.append(i)

            user_fav.sort(reverse=True)
            reading_list=[]
            for i in range(min(len(user_fav),10)):
                reading_list.append(user_fav[i])

        else:
            print("User has not reviewed any book")

In [85]:
for x in reading_list:
    for i in temp_user:
        for j in list(G.neighbors(i)): 
            if G.has_edge(i,x) and G.has_edge(i,j) and j!=x:
                wt[j][x]=wt[j][x]+1
                Z.add_node(j, node_weight=G.degree(j,weight='weight')/G.degree(j))
                Z.add_node(x, node_weight=G.degree(x,weight='weight')/G.degree(x))
                Z.add_edge(j,x, weight=wt[j][x])
                
Z.add_node(current_node)

In [86]:
reco=[] #recommendation list

#finding the best similar books
if flag1:
    for i in list(Z.neighbors(current_node)):
        temp=[]
        temp.append(i)
        temp.append(Z.node[i]['node_weight']*Z.edges[current_node,i]['weight'])
        reco.append(temp)

    reco.sort(key=lambda x:x[1], reverse=True) #sorting the recommendation list
    if not reco:
        reco1=random.sample(list(Z), min(nx.number_of_nodes(Z),5))
    print(f'{Fore.BLUE}You are currently viewing:{Style.RESET_ALL}')
    print(Books.loc[Books['count'] == current_node, 'bookTitle'])
    print(f'{Fore.RED}We recommend that you read:{Style.RESET_ALL}')

    if not reco:
        for i in reco1:
            print(Books.loc[Books['count'] == i, 'bookTitle'])
        
    else:
        for i in range(min(len(reco),reco_number)):
            print(Books.loc[Books['count'] == reco[i][0], 'bookTitle'])

#The following is the output if you have selected recommendations for currently viewing book
#If you chose option 1, you do not have to run the next set of code

In [89]:
#common neighbors    
temp=reading_list
for i in temp:
    if i not in Z.nodes():
        temp.remove(i)

to_read1=list(set(Z.neighbors(temp[0])))

for i in temp:
    to_read1=list(set(Z.neighbors(i)).intersection(to_read1))

In [90]:
#common shortest path

#finding shortest paths amaong the reading_list books
tempx=[]
for x in reading_list:
    for y in reading_list:
        if G.has_node(x) and G.has_node(y):
            if nx.has_path(G,x,y):
                temp=list(nx.shortest_path(G, source=x, target=y))
                tempx.append(temp)
to_read2=list(set(tempx[0]).intersection(tempx[1]))

#finding common elements in the shortest paths

for i in range(len(tempx)):
    to_read2=list(set(tempx[i]).intersection(to_read2))

to_read=list(set(to_read1) | set(to_read2))
    
if not to_read:
    to_read=random.sample(list(Z), min(nx.number_of_nodes(Z),2))

for i in reading_list:
    if i in to_read:
        to_read.remove(i)

In [91]:
print(f'{Fore.BLUE}Your reading list is:{Style.RESET_ALL}')
for i in reading_list:
    print(Books.loc[Books['count'] == i, 'bookTitle'])
print(f'{Fore.RED}From observations we made on your reading list, we recommend you read:{Style.RESET_ALL}')
for i in range(min(len(to_read),10)):
    print(Books.loc[Books['count'] == to_read[i], 'bookTitle'])

#The following is the output for your reading list or the user's reading list

Your reading list is:
248974    Por aqui nÃ£o Ã© passagem: Contos
Name: bookTitle, dtype: object
248971    Garzanti - Gli Elefanti: Dolori Del Giovane We...
Name: bookTitle, dtype: object
248969    El Lobo de Mar
Name: bookTitle, dtype: object
248967    Jane Eyre
Name: bookTitle, dtype: object
248962    Allah muss nicht gerecht sein.
Name: bookTitle, dtype: object
248957    Der Ferienkrimi. Ein mÃ?Â¶rderischer Sommer.
Name: bookTitle, dtype: object
248949    Felix und das liebe Geld
Name: bookTitle, dtype: object
248944    Un peu plus loin sur la droite
Name: bookTitle, dtype: object
248943    Le Silkie
Name: bookTitle, dtype: object
248942    Edge #3
Name: bookTitle, dtype: object
From observations we made on your reading list, we recommend you read:
32440    Tending Roses
Name: bookTitle, dtype: object
81596    The life of Sir Arthur Conan Doyle
Name: bookTitle, dtype: object
32454    Blue Diary
Name: bookTitle, dtype: object
16079    Lullaby: A Novel
Name: bookTitle, dtype: object
3